In [12]:
import websocket
import json
import gym
from gym import spaces
import numpy as np
import numexpr as ne

In [13]:
websocket.enableTrace(True)
ws=websocket.WebSocket()
ws.connect("ws://10.42.0.231:80/")

--- request header ---
GET / HTTP/1.1
Upgrade: websocket
Host: 10.42.0.231
Origin: http://10.42.0.231
Sec-WebSocket-Key: S7Opfo7QrO2vKJ8ueVMADA==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Server: arduino-WebSocketsServer
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Version: 13
Sec-WebSocket-Accept: MMoTmSOWyrsj+xOwsMVnwIWn3oU=
-----------------------


In [56]:
range(0, 100)

range(0, 100)

In [60]:
import numpy as np
np.array([[1,2,3],[1,2,3]]).shape

(2, 3)

In [7]:
import websocket
import json
import gym
from gym import spaces
import numpy as np
import re
import numexpr as ne
import time
from collections import OrderedDict
import logging
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly
import json
import os
plotly.io.json.config.default_engine = 'orjson'

#steps are not implemented for now
out_dict={'leddelay':{'names':['leddelay'], 'value_range':{'min':1, 'max':10001, 'step':100}, 'init_val':{'leddelay':10}},
          'ledcontrols':{'names':['lv1r','lv1g','lv1b','lv2r','lv2g','lv2b','lv3r','lv3g','lv3b','lv4r','lv4g','lv4b', 'lv5r','lv5g','lv5b','lv6r','lv6g',
          'lv6b',
          'lv7r',
          'lv7g',
          'lv7b',
          'lv8r',
          'lv8g',
          'lv8b'], 'value_range':{'min':10, 'max':255, 'step':10}, 'init_val':{'lv1r':255, 'lv1g':0, 'lv1b':0, 
                                                                               'lv2r':0, 'lv2b':255, 'lv2b':0, 
                                                                               'lv3r':0, 'lv3g':0, 'lv3b':255, 
                                                                               'lv4r':255, 'lv4g':255, 'lv4b':255, 
                                                                               'lv5r':0,   'lv5g':255, 'lv5b':255, 
                                                                               'lv6r':255, 'lv6g':0,   'lv6b':255, 
                                                                               'lv7r':255, 'lv7b':255, 'lv7b':0, 
                                                                               'lv8r':255, 'lv8g':0, 'lv8b':0}},
          'sound_wave_frequencies':{'names':['wave_1_freq','wave_2_freq'], 'value_range':{'min':1, 'max':30000, 'step':100}, 'init_val':{'wave_1_freq':440, 
                                                                                                                                         'wave_2_freq':440}},
          'panner_phasor_frequencies':{'names':['panner_freq', 'phasor_1_freq', 'phasor_2_freq','phasor_1_min',  'phasor_2_min', 'phasor_1_dif', 'phasor_2_dif'],  'value_range':{'min':1, 'max':50, 'step':1},
                                       'init_val':{'panner_freq':1,
                                                    'phasor_1_freq':10,
                                                    'phasor_2_freq':10,
                                                    'phasor_1_min':1,
                                                    'phasor_2_min':1,
                                                    'phasor_1_dif':30,
                                                    'phasor_2_dif':30}},
          'panner_div':{'names':['panner_div'], 'value_range':{'min':1, 'max':5, 'step':1}, 'init_val':{'panner_div':2}},
          'sound_wave_shapes':{'names':['wave_1_type', 'wave_2_type'], 'value_range':{'min':0, 'max':3, 'step':1}, 
                               'init_val':{'wave_1_type':0,
                                           'wave_2_type':0}},
          'maxibolume':{'names':['maxivolume'], 'value_range':{'min':0, 'max':50, 'step':10}, 
                        'init_val':{'maxivolume':10}}
}
out_order=['lv1r','lv1g','lv1b','lv2r','lv2g','lv2b','lv3r','lv3g','lv3b','lv4r','lv4g','lv4b','lv5r','lv5g','lv5b','lv6r','lv6g','lv6b',
          'lv7r','lv7g','lv7b','lv8r','lv8g','lv8b','leddelay','wave_1_freq','wave_2_freq','panner_freq','panner_div','phasor_1_freq',
          'phasor_1_min','phasor_1_dif','phasor_2_freq','phasor_2_min','phasor_2_dif','maxivolume','wave_1_type','wave_2_type']


class SFSystemCommunicator(gym.Env):
    def __init__(self, out_dict=out_dict, out_order=out_order,n_input_channels=8, channels_of_interest_inds=list(range(8)), n_timepoints_per_sample=100, max_sfsystem_output=1023,reward_formula_string='(fbin_05_4_ch0+freq_30_ch0)/fbin_12_30_ch0', 
                 fbins=[(0.001, 0.5), (0.5, 4), (4,8), (8,12), (12,30)], delay=10,
                 use_raw_in_os_def=False, use_freq_in_os_def=False, use_fbins_in_os_def=False, device_address="ws://10.42.0.231:80/",
                 step_stim_length_millis=10000, episode_time_seconds=60, collect_data_toplot=True, render_data=True, return_plotly_figs=False,
                 logfn='current_training.log', log_steps=True, log_episodes=True, log_best_actions_final=True, signal_plot_width=2000, signal_plot_height=1500, training_plot_width=2000, training_plot_height=500, 
                 write_raw=True,
                 write_fft=True,
                 write_bins=True,
                 log_best_actions_every_episode=True,
                 log_actions_every_step=True):
        self.device_address=device_address
        self.step_stim_length=step_stim_length_millis/1000
        self.episode_time_seconds=episode_time_seconds
        self.n_steps_per_episode=int(self.episode_time_seconds*1000/self.step_stim_length)
        self.cur_step=0
        self.render_data=render_data
        self.return_plotly_figs=return_plotly_figs
        
        self.out_dict=out_dict
        self.out_order=out_order
        self.max_sfsystem_output=max_sfsystem_output
        self.n_timepoints_per_sample=n_timepoints_per_sample
        self.n_input_channels=n_input_channels
        self.reward_formula_string=reward_formula_string

        self.delay=delay

        self.record_raw=use_raw_in_os_def

        self.do_fft=use_freq_in_os_def
        self.record_fft=use_freq_in_os_def

        self.do_fbins=use_fbins_in_os_def
        self.record_fbins=use_fbins_in_os_def

        self.fbins=fbins
        self.n_fbins=len(self.fbins)

        self.channels_of_interest_inds=channels_of_interest_inds
        self.channels_of_interest_inds.sort()
        self.n_channels_of_interest=len(self.channels_of_interest_inds)

        self.set_fft_params()

        self.timesleep_period=0.1

        if 'raw' in self.reward_formula_string:
            self.record_raw=True
        if 'freq' in self.reward_formula_string:
            self.record_fft=True
        if 'fbin' in self.reward_formula_string:
            self.record_fbins=True

        if self.record_fbins:
            self.do_fft=True
            self.do_fbins=True
        
        self.init_action_space()
        self.init_observation_space()
        self.set_value_dict_for_reward_function()
        self.connect()
        print(self.connection_status)
        self.set_delay_and_data_transfer_buffer_size()
        print('Delay and data transfer buffer size are set up.')
        self.set_default_actions()
        print('Default actions are set.')


        #logging part
        self.logfn=logfn
        self.collect_data_toplot=collect_data_toplot

        self.write_raw=write_raw
        self.write_fft=write_fft
        self.write_bins=write_bins

        self.log_steps=log_steps
        self.log_episodes=log_episodes
        
 
        self.clear_reward_buffers()   
        self.clear_reward_stats()
        self.current_episode=0
        self.set_fbin_x_axis_labels()
        
        logging.basicConfig(filename=self.logfn, level=logging.INFO)
        self.signal_plot_width=signal_plot_width
        self.signal_plot_height=signal_plot_height
        self.training_plot_width=training_plot_width
        self.training_plot_height=training_plot_height
        #action logging
        self.best_action_overall=None
        self.best_action_episode=None
        self.log_actions_every_step=log_actions_every_step
        self.log_best_actions_every_episode=log_best_actions_every_episode
        self.log_best_actions_final=log_best_actions_final


    def help(self):
        print('Reward formula can use the following operators: //, *, **, -, +')
        print('It can refer to channels using "ch" prefix followed by an index (starting with 0) e.g. ch0')
        print('It can refer to values of frequency bins in specific channels e.g. fbin_10_50_ch0')
        print('The corresponding freqency bins must be present among fbins passed at the initialization step')
        print('fbins should be passed in the form of [(b1 min, b1max),...(bn min, bn max)]')
        print('It can refer to specific frequencies from fft e.g. freq_50_ch0')
        print('For fractional frequencies (e.g. 0.5) do not use points, just start with 0 (e.g. 05)')
        print('Some examples:')
        print('1. (freq_50_ch0+fbin_0_10_ch0)/(fbin_20_30_ch0)')
        print('2. freq_5_ch0/freq_10_ch0')
        print('3. fbin_05_5_ch0')
    def clear_log(self):
        if os.path.isfile(self.logfn):
            os.remove(self.logfn)
    def set_value_dict_for_reward_function(self):
        ftokens=re.split(r'[+/)(*]+',self.reward_formula_string)
        self.rewarddict={}
        self.tokendict={}
        for token in ftokens:
            if 'ch' in token:
                self.rewarddict[token]=None
                self.tokendict[token]={}
                subtokens=token.split('_')
                self.tokendict[token]['datatype']=subtokens[0]
                for subtoken in subtokens:
                    if 'ch' in subtoken:
                        self.tokendict[token]['channelindex']=int(subtoken.split('h')[1])
                    if subtoken=='freq':
                        tfreq=float(subtokens[1])
                        self.tokendict[token]['freqdata']=tfreq
                        closestind=np.argmin(np.abs(self.f_plot - tfreq))
                        self.tokendict[token]['closest_fft_ind']=closestind
                        print(f'Token {token}:')
                        print(f'Closest fft frequency {self.f_plot[closestind]}')
                    if subtoken=='fbin':
                        bin_lst=[subtokens[1],subtokens[2]]
                        for i in range(2):
                            val=bin_lst[i]
                            if val.startswith('0'):
                                val=float('0.'+val[1:])
                            else:
                                val=float(val)
                            bin_lst[i]=val
                        self.tokendict[token]['freqdata']=tuple(bin_lst)
                        self.tokendict[token]['fbin_idx']=self.fbins.index(self.tokendict[token]['freqdata'])
    def set_default_actions(self):
        observation_space_sample=self.observation_space.sample()
        for key1 in observation_space_sample:
            for key2 in self.out_dict:
                if key1 in self.out_dict[key2]['init_val']:
                    observation_space_sample[key2]=self.out_dict[key1]['init_val'][key2]

        self.default_actions=observation_space_sample
    def populate_rewarddict(self, observations):
        for token, tokendata in self.tokendict.items():
            tartype=tokendata['datatype']
            tarchannelidx=tokendata['channelindex']
            if tartype=='raw':
                tarobs=observations['raw_data']
                tarobs=tarobs[:,tarchannelidx]
                res=np.mean(tarobs) #here we use the mean, but this may be changed
            if tartype=='freq':
                #tarfreq=tokendata['freqdata']
                taridx=tokendata['closest_fft_ind']
                tarobs=observations['fft'][tarchannelidx]
                res=tarobs[taridx]
            if tartype=='fbin':
                tarobs=observations['fbins'][tarchannelidx]
                res=tarobs[tokendata['fbin_idx']]
            self.rewarddict[token]=res
    def get_reward(self, observations=None, toreturn=False):
        if str(observations) != "None":
            observations=self.observation_space.sample() #if no observations are given extrernally, sample from the observation space
        self.populate_rewarddict(observations)
        self.reward=ne.evaluate(self.reward_formula_string, local_dict=self.rewarddict)
        if toreturn==True:
            return self.reward
    def init_action_space(self):
        self.action_space=spaces.Dict({})
        for key, val in self.out_dict.items():
            spacesnames=val['names']
            spacesrange=val['value_range']
            for spacename in spacesnames:
                self.action_space.spaces[spacename]=spaces.Box(low=spacesrange['min'], high=spacesrange['max'], shape=(1,), dtype=int)
    def init_observation_space(self):
        self.observation_space=spaces.Dict({})
        self.observation_space['raw_data']=spaces.Box(low=0, high=self.max_sfsystem_output, shape=(self.n_timepoints_per_sample, self.n_channels_of_interest), dtype=int) #n timepoints per sample rows, n input channels columns, signals should be normalized
        if self.record_fft:
            self.observation_space['fft']=spaces.Box(low=0.0, high=1.0, shape=(self.n_channels_of_interest, self.n_fft_values))
        if self.record_fbins:
            self.observation_space['fbins']=spaces.Box(low=0.0, high=1.0, shape=(self.n_channels_of_interest, self.n_fbins))
    def set_fft_params(self):
        self.sampling_frequency=int(1000/self.delay)
        self.max_possible_fft_frequency=self.sampling_frequency/2
        self.sampling_period=int(self.delay*self.n_timepoints_per_sample)
        self.timesteps=np.linspace(0, (self.n_timepoints_per_sample-1)*self.delay, self.n_timepoints_per_sample)
        self.fstep=self.sampling_frequency/self.n_timepoints_per_sample
        self.f=np.linspace(0, (self.n_timepoints_per_sample-1)*self.fstep, self.n_timepoints_per_sample)
        self.f_plot=self.f[0:int(self.n_timepoints_per_sample/2 + 1)]
        self.n_fft_values=len(self.f_plot)

    def connect(self):
        websocket.enableTrace(True)
        self.ws=websocket.WebSocket()
        self.ws.connect(self.device_address)
        self.connection_status=self.ws.recv()
    
    def get_fft_fromsignal(self, raw_singlech):
        X=np.fft.fft(raw_singlech)
        X_mag=np.abs(X)/self.n_timepoints_per_sample
        X_mag_plot=2*X_mag[0:int(self.n_timepoints_per_sample/2 + 1)]
        return X_mag_plot

    def get_fft_allchannels(self, raw_data):
        if str(raw_data) != "None":
            raw_data=self.observation_space.sample()['raw_data']

        fft_data=[]
        for chindex in range(raw_data.shape[1]):
            chraw=raw_data[:,chindex]
            chfft=self.get_fft_fromsignal(chraw)
            fft_data.append(chfft)
        fft_data=np.array(fft_data)
        return fft_data
    
    def get_bin_values_from_signal(self, fft_signlech):
        fpl=np.array(self.f_plot)
        xmp=np.array(fft_signlech)
        magnitudes=[]
        for low, high in self.fbins:
            mask = (fpl >= low) & (fpl < high)
            magnitude = np.abs(xmp[mask]).mean() #here can be other functions
            magnitudes.append(magnitude)
        magnitudes=np.array(magnitudes)
        return magnitudes
    def set_fbin_x_axis_labels(self):
        self.fbin_axis_labels=[]
        for low, high in self.fbins:
            binname=f'{low}-{high} Hz'
            self.fbin_axis_labels.append(binname)
    def get_bin_values_allchannels(self, fft=None):
        if str(fft) != "None":
            fft=self.observation_space.sample()['fft']
        fbins_data=[]
        for chindex in range(fft.shape[0]):
            chfft=fft[chindex,:]
            chbins=self.get_bin_values_from_signal(chfft)
            fbins_data.append(chbins)
        fbins_data=np.array(fbins_data)
        return fbins_data

    def set_delay_and_data_transfer_buffer_size(self):
        self.ws.send("set_delay_and_data_transfer_buffer_size")
        time.sleep(self.timesleep_period)
        setup=False
        while setup==False:
            try:
                device_msg=self.ws.recv()
                if device_msg == "Awaiting delay and data transfer buffer size in shape with space separator":
                    self.ws.send(f'{self.delay},{self.n_timepoints_per_sample}')
                time.sleep(self.timesleep_period)
                device_msg=self.ws.recv()
                if device_msg == "Delay and data transfer buffer size set up":
                    setup = True 
                    break;
            except:
                pass
    def start_data_transfer_from_device(self):
        self.ws.send("start_data_transfer_from_ads")
    def stop_data_transfer_from_device(self):
        self.ws.send("stop_data_transfer_from_ads")
    def stop_audiovis_feedback(self):
        self.ws.send("stop_led_cycle")
    def update_audiovis_feedback(self, update_dict=None):
        if str(update_dict)!= 'None':
            update_dict=self.action_space.sample()
        self.ws.send("receive_output_control_data")
        outmsg_vals=[]
        for controlnm in self.out_order:
            outmsg_vals.append(update_dict[controlnm][0])
        self.current_control_msg=','.join(list(map(str,outmsg_vals)))
        self.ws.send(self.current_control_msg)
    def sample_observations(self):
        self.ws.send("start_data_transfer_from_ads")
        self.current_sample=json.loads(self.ws.recv())
        self.raw_data=[]
        for key, value in self.current_sample.items():
            self.raw_data.append(value)
        self.raw_data=np.array(self.raw_data).transpose()
        self.raw_data=self.raw_data[:,self.channels_of_interest_inds]
        self.ws.send("stop_data_transfer_from_ads")
    def sample_and_process_observations_from_device(self):
        new_observations=dict()
        self.sample_observations()
        new_observations['raw_data']=self.raw_data
        if self.do_fft:
         self.fft=self.get_fft_allchannels(raw_data=self.raw_data)
         if self.record_fft:
            new_observations['fft']=self.fft
        if self.do_fbins:
            self.fbins_data=self.get_bin_values_allchannels(fft=self.fft)
            if self.record_fbins:
                new_observations['fbins']=self.fbins_data 
        new_observations=OrderedDict(new_observations)
        return new_observations
    def write_signal_logs(self):
        if self.record_fft==True:
            if self.write_fft==True:
                logging.info(json.dumps({'fft':self.cur_observations['fft'].tolist()}))
        if self.record_fbins==True:
            if self.write_bins==True:
                logging.info(json.dumps({'fbins':self.cur_observations['fbins'].tolist()}))
        if self.record_raw==True:
            if self.write_raw==True:
                logging.info(json.dumps({'raw_data':self.cur_observations['raw_data'].tolist()}))
    def step(self, action):
        self.best_overall_reward_now=False
        self.update_audiovis_feedback(action)
        time.sleep(self.step_stim_length)
        self.current_actions=action
        new_observations=self.sample_and_process_observations_from_device()
        self.cur_observations=new_observations
        reward=self.get_reward(observations=new_observations, toreturn=True)
        self.total_cur_episode_reward+=reward
        if reward>self.episode_max_reward:
                self.episode_max_reward=reward
                self.best_episode_reward_now=True
                self.best_action_episode=action
        if reward>self.overall_max_reward:
                self.overall_max_reward=reward
                self.best_overall_reward_now=True
                self.best_action_overall=action
        if self.total_cur_episode_reward>self.total_episode_max_reward:
            self.total_episode_max_reward=self.total_cur_episode_reward
            self.best_total_episode_reward_now=True

        if self.collect_data_toplot:
            self.cur_episode_rewards.append(reward)
        if self.log_steps:
            logging.info(json.dumps({'Episode':self.current_episode, 'Step': self.cur_step, 'Step reward': reward}))
        if self.log_actions_every_step:
            logging.info(json.dumps({'Action reward':reward}))
            actionstring=self.get_json_string_from_ordered_dict(action)
            logging.info(actionstring)
        self.write_signal_logs()        
              


        if self.cur_step<self.n_steps_per_episode:
            done=False
            self.cur_step+=1
        else:
            done=True
        return new_observations, reward, done, {}
    def reset(self):
        if self.collect_data_toplot:
            self.previous_episodes_total_rewards.append(self.total_cur_episode_reward)
            self.previous_episodes_max_rewards.append(self.episode_max_reward)
        if self.log_episodes:
            logging.info(json.dumps({'Episode':self.current_episode, 'Episode total reward': self.total_cur_episode_reward, 'Episode max reward': self.episode_max_reward}))
        if self.log_best_actions_every_episode:
            actionstring=self.get_json_string_from_ordered_dict(self.best_action_episode)
            logging.info(json.dumps({'Best action in the episode reward':self.episode_max_reward}))
            logging.info(actionstring)
        self.stop_audiovis_feedback()
        self.cur_step=0
        self.episode_max_reward=0
        self.total_cur_episode_reward=0

        self.best_episode_reward_now=False #just in case
        self.best_overall_reward_now=False #just in case
        self.best_total_episode_reward_now=False #just in case

        self.best_action_epoch=None

        new_observations=self.sample_and_process_observations_from_device()
        self.cur_observations=new_observations
        self.update_audiovis_feedback(update_dict=self.default_actions) #update back to default actions
        if self.collect_data_toplot==True:
            self.previous_episodes_max_rewards.append(self.episode_max_reward)
            self.cur_episode_rewards=[]
        time.sleep(self.step_stim_length) #prepare the brain for the next episode 
        return new_observations

    def get_json_string_from_ordered_dict(self, od):
        od=dict(od)
        for key, value in od.items():
            od[key]=value.tolist()
        return json.dumps(od)

    def render(self, elems=['reward_lineplots', 'current_fft', 'current_fbins'], return_figs=None):
        if return_figs==None:
            return_figs=self.return_plotly_figs
        figures=dict()
        if 'reward_lineplots' in elems:
            training_fig=sp.make_subplots(rows=2, cols=2)
            training_fig.update_layout(width=self.training_plot_width, height = self.training_plot_height)
            training_fig.add_trace(sp.go.Scatter(x=list(range(len(self.cur_episode_rewards))), y=self.cur_episode_rewards, mode='lines+markers', name='Current episode rewards'), row=1, col=1)
            training_fig.add_trace(sp.go.Scatter(x=list(range(len(self.previous_episodes_max_rewards))), y=self.previous_episodes_max_rewards, mode='lines+markers', name='Previous episode max rewards'), row=1, col=2)
            training_fig.add_trace(sp.go.Scatter(x=list(range(len(self.previous_episodes_total_rewards))), y=self.previous_episodes_total_rewards, mode='lines+markers', name='Previous episode total rewards'), row=2, col=1)           
            figures['reward_lineplots']=training_fig
            if self.render_data:
                training_fig.show()
        if ('current_fft' in elems) or ('current_fbins' in elems):
            if self.record_fft or self.record_fbins:
                signal_fig=sp.make_subplots(rows=self.n_channels_of_interest, cols=2)
                signal_fig.update_layout(width=self.signal_plot_width, height = self.signal_plot_height)
                if 'current_fft' in elems and self.record_fft:
                    for chidx in range(self.n_channels_of_interest):
                        orig_chidx=self.channels_of_interest_inds[chidx]
                        chfft=self.cur_observations['fft'][chidx]
                        signal_fig.add_trace(sp.go.Scatter(x=self.f_plot, y=chfft, mode='lines+markers', name=f'Channel {orig_chidx} spectrum'), row=chidx+1, col=1)
                if 'current_fbins' in elems and self.record_fbins:
                    for chidx in range(self.n_channels_of_interest):
                        orig_chidx=self.channels_of_interest_inds[chidx]
                        chbins=self.cur_observations['fbins'][chidx]
                        signal_fig.add_trace(sp.go.Bar(x=self.fbin_axis_labels, y=chbins, name=f'Channel {orig_chidx} frequency bins'), row=chidx+1, col=2)
                if self.render_data:
                    signal_fig.show()
        if return_figs:
            return figures
    def clear_reward_buffers(self):
        self.cur_episode_rewards=[]
        self.previous_episodes_max_rewards=[]
        self.previous_episodes_total_rewards=[]
    def clear_reward_stats(self):
        if self.log_best_actions_final:
            actionstring=self.get_json_string_from_ordered_dict(self.best_action_overall)
            logging.info(json.dumps({'Best action across episodes reward':self.overall_max_reward}))
            logging.info(actionstring)
        self.episode_max_reward=0
        self.overall_max_reward=0
        self.total_cur_episode_reward=0
        self.total_episode_max_reward=0
        self.best_episode_reward_now=False
        self.best_overall_reward_now=False
        self.best_total_episode_reward_now=False
    def close(self, clear_log=False):
        self.stop_audiovis_feedback() #just in case
        self.stop_data_transfer_from_device() #just in case
        self.cur_step=0 #just in case
        self.current_episode=0

        self.clear_reward_buffers()
        self.clear_reward_stats()
        if clear_log:
            self.clear_log()
        self.ws.close()
        
        

In [128]:
import torch
import torch.optim as optim
class trainer():
    def __init__(self, environment, agent, optimizer, optim_params={'lr':0.001}, num_episodes=1000, log_model=True, render=True, get_plots=False):
        self.env=environment()
        self.agent=agent(self.env)
        self.optim_params=optim_params
        self.optim=optimizer(self.agent.model.parameters(), lr=self.optim_params['lr'])
        self.num_episodes=num_episodes
        self.log_model=log_model
        self.render=render
        self.get_plots=get_plots
    def train(self, num_episodes=None, render=None, get_plots=None):
        if str(num_episodes)=='None':
            num_episodes=self.num_episodes
        if str(render)=='None':
            render=self.render
        if str(get_plots)=='None':
            get_plots=self.get_plots
        for episode in range(num_episodes):
            state=self.env.reset()
            done=False
            while not done:
                action=self.agent.select_action(state)
                next_state, reward, done, _=self.env.step(action)
                loss = self.agent.update_policy(state, action, reward, next_state, done)
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                state=next_state
                #best_episode_reward_now=self.env.best_episode_reward_now
                #best_overall_reward_now=self.env.best_overall_reward_now
                #best_total_episode_reward_now=self.env.best_total_episode_reward_now
                #self.overall_max_reward=self.env.overall_max_reward
                #self.total_episode_max_reward=self.env.total_episode_max_reward
                #self.total_cur_episode_reward=self.env.total_cur_episode_reward
                #self.episode_max_reward=self.env.episode_max_reward
                
                if self.log_model==True:
                    if self.env.best_overall_reward_now:
                        torch.save(self.agent.model.state_dict(), 'model_best_overall_reward.pth')
                    if self.env.best_total_episode_reward_now:
                        torch.save(self.agent.model.state_dict(), 'model_best_total_episode_reward.pth')

                if render:
                    self.env.render(return_figs=get_plots)
            

In [3]:
communicator=SFSystemCommunicator()

--- request header ---
GET / HTTP/1.1
Upgrade: websocket
Host: 10.42.0.231
Origin: http://10.42.0.231
Sec-WebSocket-Key: F5YK/4GjCUGdzYIbNwn0NA==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
--- response header ---


HTTP/1.1 101 Switching Protocols
Server: arduino-WebSocketsServer
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Version: 13
Sec-WebSocket-Accept: DvfFGRCrIVo0MZ2JY6Z6/2OWxZo=
-----------------------
++Rcv raw: b'\x89\x00'
++Rcv decoded: fin=1 opcode=9 data=b''
++Sent raw: b'\x8a\x80\xbe\x13\xf9\xbc'
++Sent decoded: fin=1 opcode=10 data=b''
++Rcv raw: b'\x81\tConnected'
++Rcv decoded: fin=1 opcode=1 data=b'Connected'
++Sent raw: b'\x81\xa7\x89bT\xd9\xfa\x07 \x86\xed\x078\xb8\xf0=5\xb7\xed=0\xb8\xfd\x03\x0b\xad\xfb\x03:\xaa\xef\x07&\x86\xeb\x172\xbf\xec\x10\x0b\xaa\xe0\x181'
++Sent decoded: fin=1 opcode=1 data=b'set_delay_and_data_transfer_buffer_size'
++Rcv raw: b'\x81JAwaiting delay and data transfer buffer size in shape with space separator'
++Rcv decoded: fin=1 opcode=1 data=b'Awaiting delay and data transfer buffer size in shape with space separator'
++Sent raw: b'\x81\x86p\x1f\xa8\xdbA/\x84\xea@/'
++Sent decoded: fin=1 opcode=1 data=b'10,100'


Token freq_30_ch0:
Closest fft frequency 30.0
Connected


++Rcv raw: b'\x81*Delay and data transfer buffer size set up'
++Rcv decoded: fin=1 opcode=1 data=b'Delay and data transfer buffer size set up'


Delay and data transfer buffer size are set up.
Default actions are set.


In [126]:
communicator.collect_data_toplot

True

In [111]:
communicator.write_log

True

In [129]:
communicator.close()

++Sent raw: b'\x81\x8e\xb3\xb2\xf4\xe7\xc0\xc6\x9b\x97\xec\xde\x91\x83\xec\xd1\x8d\x84\xdf\xd7'
++Sent decoded: fin=1 opcode=1 data=b'stop_led_cycle'
++Sent raw: b'\x81\x9bo[\xbe}\x1c/\xd1\r0?\xdf\t\x0e\x04\xca\x0f\x0e5\xcd\x1b\n)\xe1\x1b\x1d4\xd3"\x0e?\xcd'
++Sent decoded: fin=1 opcode=1 data=b'stop_data_transfer_from_ads'
++Sent raw: b'\x88\x82\x93\x8c\xd7\xb8\x90d'
++Sent decoded: fin=1 opcode=8 data=b'\x03\xe8'


In [82]:
communicator.reward

array(5.275713, dtype=float32)

In [131]:
res=communicator.step(communicator.action_space.sample())

++Sent raw: b'\x81\x9bA;\xba+3^\xd9N(M\xdft.N\xce[4O\xe5H.U\xceY.W\xe5O O\xdb'
++Sent decoded: fin=1 opcode=1 data=b'receive_output_control_data'
++Sent raw: b'\x81\xfe\x00\x84\xd1\xd2\x8e\xaa\xe3\xe3\xb9\x86\xe3\xe1\xba\x86\xe0\xe4\xbb\x86\xe0\xea\xa2\x9b\xe9\xe0\xa2\x9e\xe3\xfe\xbd\x93\xfd\xe3\xbd\x92\xfd\xe3\xb6\x86\xe6\xe4\xa2\x9c\xe2\xfe\xbf\x9d\xfd\xe3\xb7\x9b\xfd\xe3\xbc\x9a\xfd\xe3\xbe\x98\xfd\xe0\xbc\x9c\xfd\xe0\xbb\x9f\xfd\xe0\xbb\x9e\xfd\xe6\xb6\x86\xe0\xe6\xbf\x86\xe9\xe2\xa2\x9b\xe2\xe5\xa2\x98\xe1\xe5\xa2\x9b\xe9\xea\xa2\x98\xe3\xe5\xb9\x86\xe3\xeb\xbd\x9e\xe6\xfe\xbf\x9b\xe6\xe6\xbc\x86\xe0\xe7\xa2\x9f\xfd\xe6\xbe\x86\xe3\xe1\xa2\x9e\xe7\xfe\xb6\x86\xe3\xe4\xa2\x98\xe5\xfe\xbd\x98\xfd\xe1\xa2\x9b'
++Sent decoded: fin=1 opcode=1 data=b'217,234,165,18,182,42,39,138,18,76,63,17,191,120,102,226,255,254,48,141,80,137,207,188,2277,29347,11742,15,5,40,23,46,8,26,24,32,3,1'
++Sent raw: b'\x81\x9c\xe1P>\xd9\x92$_\xab\x95\x0fZ\xb8\x951a\xad\x931P\xaa\x875L\x86\x87"Q\xb4\xbe1Z\xaa'

here


/tmp/ipykernel_29567/4154560659.py:274: RuntimeWarning:

Mean of empty slice.

/home/biorp/.local/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning:

invalid value encountered in divide



In [137]:
str(communicator.cur_observations['raw_data'])

'[[0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\n [0 0 0 0 0 0 0 0]\

In [147]:
s=json.dumps({'fft':communicator.cur_observations['fft'].tolist()})
json.loads(s)

{'fft': [[1061.82,
   64.85400006812361,
   32.05495321370078,
   37.45840095159833,
   40.12737658646226,
   71.23357224860794,
   82.65749945364101,
   86.90947544120378,
   56.68757962490529,
   72.66634427665639,
   89.94266372698853,
   29.74025383177499,
   43.97035613047283,
   67.9856736888025,
   144.40794694960115,
   19.744174774811526,
   49.4974469963815,
   34.67174799366255,
   24.311509411140914,
   17.013056768317995,
   36.899002180445834,
   39.36445239701676,
   67.37027060991637,
   44.49914624314557,
   50.192479732744275,
   123.83855457812808,
   70.87210609941444,
   8.251014486548272,
   4.514411299458669,
   74.31174780532974,
   49.1794392169519,
   28.522124232046686,
   55.6631156419679,
   37.522468137469,
   36.24489987496416,
   43.563960177541865,
   54.48147322683956,
   12.761606074613063,
   68.65836115023832,
   32.76670651208926,
   108.77349143098908,
   62.114658938899886,
   7.716727266762501,
   110.02813722740102,
   59.42707236476817,
   47.

In [132]:
communicator.render()

In [80]:
res

(OrderedDict([('raw_data',
               array([[0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0, 0],
             

In [73]:
for low, high in communicator.fbins:
    print(low, high)

ValueError: too many values to unpack (expected 2)

In [74]:
communicator.fbins

array([[       nan, 0.6982351 , 0.6538787 , 0.43219435, 0.5053414 ],
       [       nan, 0.5992771 , 0.6171261 , 0.3235731 , 0.42574465],
       [       nan, 0.29284555, 0.64092684, 0.77488357, 0.47472328],
       [       nan, 0.50798297, 0.46259686, 0.5543925 , 0.5086066 ],
       [       nan, 0.45116618, 0.48398477, 0.79926497, 0.5995662 ],
       [       nan, 0.5164592 , 0.33149385, 0.5761446 , 0.53982437],
       [       nan, 0.31781006, 0.37277874, 0.52640885, 0.5283958 ],
       [       nan, 0.3079564 , 0.6230651 , 0.31849974, 0.5566319 ]],
      dtype=float32)

In [72]:
communicator.fft

array([[ 977.94      ,   94.79825768,   35.85994414,  116.29147161,
          58.17282561,   35.85678481,   15.70999903,   56.10109109,
          90.15924147,   60.17444454,   18.45576598,  117.7905015 ,
          42.8311498 ,   54.48442461,   73.40662573,   64.59555359,
          40.2182514 ,   18.46259997,   18.2396288 ,   10.72506476,
          59.97846249,   33.57476122,   92.8180531 ,   20.58441116,
          35.45424914,   31.57437569,   20.30761684,   48.4961765 ,
         159.04429771,   70.21094093,   49.69210704,   18.81233736,
          92.78241405,   48.17055463,   51.66721257,   68.85679848,
          71.76493153,   46.09494315,   14.85317015,   79.06896782,
          63.247571  ,   50.19697439,   78.05301918,   47.09011193,
          66.83176778,   62.76192749,   63.72843927,   92.15048586,
          46.31383209,   38.68174902,   35.62      ],
       [ 949.5       ,  109.48324881,   50.83762211,   64.38010751,
          59.64524474,   18.4722154 ,   29.65695507,   53.7030

In [59]:
communicator.raw_data.shape

(100, 8)

In [51]:
communicator.raw_data

In [31]:
communicator.sample_observations()

++Sent raw: b'\x81\x9c\xac\xc1\xf3\xec\xdf\xb5\x92\x9e\xd8\x9e\x97\x8d\xd8\xa0\xac\x98\xde\xa0\x9d\x9f\xca\xa4\x81\xb3\xca\xb3\x9c\x81\xf3\xa0\x97\x9f'
++Sent decoded: fin=1 opcode=1 data=b'start_data_transfer_from_ads'
++Rcv raw: b'\x81~\x06\x81{"ch1":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch2":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch3":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch4":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
communicator.close()

++Sent raw: b'\x81\x8e\x88\xae\x1c\xfb\xfb\xdas\x8b\xd7\xc2y\x9f\xd7\xcde\x98\xe4\xcb'
++Sent decoded: fin=1 opcode=1 data=b'stop_led_cycle'
++Sent raw: b"\x81\x9bHUG\x8b;!(\xfb\x171&\xff)\n3\xf9);4\xed-'\x18\xed::*\xd4)14"
++Sent decoded: fin=1 opcode=1 data=b'stop_data_transfer_from_ads'
++Sent raw: b'\x88\x82\xfcci\xab\xff\x8b'
++Sent decoded: fin=1 opcode=8 data=b'\x03\xe8'


In [21]:
communicator.sample_observations()

++Sent raw: b'\x81\x9c\xfbJ\xdc\xa7\x88>\xbd\xd5\x8f\x15\xb8\xc6\x8f+\x83\xd3\x89+\xb2\xd4\x9d/\xae\xf8\x9d8\xb3\xca\xa4+\xb8\xd4'
++Sent decoded: fin=1 opcode=1 data=b'start_data_transfer_from_ads'
++Rcv raw: b'\x81~\x06\x81{"ch1":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch2":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch3":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch4":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [12]:
communicator.update_audiovis_feedback(communicator.action_space.sample())

++Sent raw: b'\x81\x9b\xd6_M\xe0\xa4:.\x85\xbf)(\xbf\xb9*9\x90\xa3+\x12\x83\xb919\x92\xb93\x12\x84\xb7+,'
++Sent decoded: fin=1 opcode=1 data=b'receive_output_control_data'
++Sent raw: b'\x81\xfe\x00\x82\xc9\xb1R\xdd\xf8\x82k\xf1\xf8\x81a\xf1\xf8\x83c\xf1\xfb\x85e\xf1\xfb\x85c\xf1\xf8\x88j\xf1\xfd\x81~\xeb\xf8\x9dd\xef\xe5\x80k\xee\xe5\x89b\xf1\xf8\x83b\xf1\xf8\x83j\xf1\xfb\x85a\xf1\xfb\x81`\xf1\xff\x81~\xe4\xf0\x9dk\xe8\xe5\x86e\xf1\xf8\x80d\xf1\xf8\x88c\xf1\xff\x85~\xe9\xfc\x9dd\xe5\xe5\x87e\xec\xf9\x9d`\xed\xf1\x82k\xf1\xfb\x86e\xec\xfe\x9d`\xec\xe5\x82~\xef\xf0\x9d`\xef\xe5\x85g\xf1\xfa\x85~\xe9\xf0\x9d`\xe5\xe5\x87~\xec\xe5\x81'
++Sent decoded: fin=1 opcode=1 data=b'139,103,121,247,241,198,40,61,62,193,80,120,128,243,202,60,99,95,77,116,191,64,45,68,6710,20839,27717,21,3,29,22,45,34,49,28,6,1,0'


In [13]:
communicator.stop_audiovis_feedback()

++Sent raw: b'\x81\x8e\xa9M\xc2\x1b\xda9\xadk\xf6!\xa7\x7f\xf6.\xbbx\xc5('
++Sent decoded: fin=1 opcode=1 data=b'stop_led_cycle'


In [8]:
communicator.current_sample

'{"ch1":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch2":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch3":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch4":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch5":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [3]:
actspace=communicator.action_space.sample()

In [14]:
communicator.rewarddict

{'fbin_05_4_ch0': None, 'freq_30_ch0': None, 'fbin_12_30_ch0': None}

In [15]:
communicator.get_reward()

In [16]:
communicator.reward

array(4.1762214, dtype=float32)

In [51]:
communicator.rewarddict

{'fbin_0.5_5_ch0': None, 'freq_30_ch0': None, 'fbin_10_20_ch0': None}

In [12]:
s = "(freq_50+fbin_0_10)/(fbin_20_30)"

# Define the variables
variables = {
    'freq_50': 10,
    'fbin_0_10': 20,
    'fbin_20_30':40
}

# Evaluate the string as a mathematical expression
result = ne.evaluate(s, local_dict=variables)
result

array(0.75)

In [3]:
communicator.print_reward_formula_legend()

Reward formula can use the following operators: //, *, **, -, +
It can refer to channels using "ch" prefix followed by an index (starting with 0) e.g. ch0
It can refer to values of frequency bins in specific channels e.g. fbin_10-50|ch0
The corresponding freqency bins must be present among fbins passed at the initialization step
fbins should be passed in the form of [(b1 min, b1max),...(bn min, bn max)]
It can refer to specific frequencies from fft e.g. freq_50|ch0
Some examples:
1. (freq_50|ch0+fbin_0-10|ch0)/(fbin_20-30|ch0)
2. freq_5|ch0/freq_10|ch0
3. fbin_0.5-5|ch0


In [14]:
communicator.connection_status

'Connected'

In [ ]:



leddelay=100
ledvalues=[30, 0, 0, 0, 255, 0, 0, 0, 255, 255, 255, 255, 0, 255, 255, 255, 0, 255, 255, 255, 0, 255, 0, 0]
sdict={'wave_1_freq':800, 'wave_2_freq':200, 'panner_freq':30, 'panner_div':2, 'phasor_1_freq':100, 'phasor_1_min':30, 'phasor_1_max':30, 
       'phasor_2_freq':30, 'phasor_2_min':30, 'phasor_2_max':30,
       'maxivolume':5,'wave_1_type':3,'wave_2_type':0}


In [15]:
ws.recv()

++Rcv raw: b'\x89\x00'
++Rcv decoded: fin=1 opcode=9 data=b''
++Sent raw: b'\x8a\x80M\xc5\x07\x01'
++Sent decoded: fin=1 opcode=10 data=b''
++Rcv raw: b'\x81\tConnected'
++Rcv decoded: fin=1 opcode=1 data=b'Connected'


'Connected'

In [16]:
ws.send("receive_output_control_data")

++Sent raw: b'\x81\x9b\xe1\xd95\xcf\x93\xbcV\xaa\x88\xafP\x90\x8e\xacA\xbf\x94\xadj\xac\x8e\xb7A\xbd\x8e\xb5j\xab\x80\xadT'
++Sent decoded: fin=1 opcode=1 data=b'receive_output_control_data'


33

In [17]:
# 0. noise
# 1. sinewave
# 2. square
# 3. triangle


leddelay=100
ledvalues=[30, 0, 0, 0, 255, 0, 0, 0, 255, 255, 255, 255, 0, 255, 255, 255, 0, 255, 255, 255, 0, 255, 0, 0]
sdict={'wave_1_freq':800, 'wave_2_freq':200, 'panner_freq':30, 'panner_div':2, 'phasor_1_freq':100, 'phasor_1_min':30, 'phasor_1_max':30, 
       'phasor_2_freq':30, 'phasor_2_min':30, 'phasor_2_max':30,
       'maxivolume':5,'wave_1_type':3,'wave_2_type':0}

In [18]:
message=','.join(list(map(str,ledvalues)))+f',{leddelay}'+f",{sdict['wave_1_freq']},{sdict['wave_2_freq']},{sdict['panner_freq']},{sdict['panner_div']},{sdict['phasor_1_freq']},{sdict['phasor_1_min']},{sdict['phasor_1_max']},{sdict['phasor_2_freq']},{sdict['phasor_2_min']},{sdict['phasor_2_max']},{sdict['maxivolume']},{sdict['wave_1_type']},{sdict['wave_2_type']}"

In [19]:
message

'30,0,0,0,255,0,0,0,255,255,255,255,0,255,255,255,0,255,255,255,0,255,0,0,100,800,200,30,2,100,30,30,30,30,30,5,3,0'

In [20]:
len(message.split(','))

38

In [21]:
nsecs=60
nmessages=int(nsecs*(1000/leddelay))
for m in range(2):
    ws.send(message)

++Sent raw: b'\x81\xf2C\x9e\x08\xb5p\xae$\x85o\xae$\x85o\xac=\x80o\xae$\x85o\xae$\x87v\xab$\x87v\xab$\x87v\xab$\x87v\xab$\x85o\xac=\x80o\xac=\x80o\xac=\x80o\xae$\x87v\xab$\x87v\xab$\x87v\xab$\x85o\xac=\x80o\xae$\x85o\xaf8\x85o\xa68\x85o\xac8\x85o\xad8\x99q\xb29\x85s\xb2;\x85o\xad8\x99p\xae$\x86s\xb2;\x85o\xab$\x86o\xae'
++Sent decoded: fin=1 opcode=1 data=b'30,0,0,0,255,0,0,0,255,255,255,255,0,255,255,255,0,255,255,255,0,255,0,0,100,800,200,30,2,100,30,30,30,30,30,5,3,0'
++Sent raw: b'\x81\xf2\xf8\xa5~@\xcb\x95Rp\xd4\x95Rp\xd4\x97Ku\xd4\x95Rp\xd4\x95Rr\xcd\x90Rr\xcd\x90Rr\xcd\x90Rr\xcd\x90Rp\xd4\x97Ku\xd4\x97Ku\xd4\x97Ku\xd4\x95Rr\xcd\x90Rr\xcd\x90Rr\xcd\x90Rp\xd4\x97Ku\xd4\x95Rp\xd4\x94Np\xd4\x9dNp\xd4\x97Np\xd4\x96Nl\xca\x89Op\xc8\x89Mp\xd4\x96Nl\xcb\x95Rs\xc8\x89Mp\xd4\x90Rs\xd4\x95'
++Sent decoded: fin=1 opcode=1 data=b'30,0,0,0,255,0,0,0,255,255,255,255,0,255,255,255,0,255,255,255,0,255,0,0,100,800,200,30,2,100,30,30,30,30,30,5,3,0'


In [22]:
ws.send('stop_led_cycle')

++Sent raw: b'\x81\x8e\xaa\x15\xa0\x99\xd9a\xcf\xe9\xf5y\xc5\xfd\xf5v\xd9\xfa\xc6p'
++Sent decoded: fin=1 opcode=1 data=b'stop_led_cycle'


20

In [263]:
len("%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d".split(','))

25